In [3]:
import sqlite3
import pandas as pd

In [21]:
import re

In [4]:
pd.set_option("display.max_colwidth",500)
pd.set_option("display.max_rows",500)

In [5]:
DB = 'sample_2018-06_1m_stateless_census_crawl.sqlite'

In [6]:
con = sqlite3.connect(DB)

In [7]:
con.row_factory = sqlite3.Row
cur = con.cursor()
js = pd.read_sql_query("SELECT * FROM javascript", con)

In [10]:
js[js.operation == "call"].symbol.value_counts().head(15)

window.Storage.getItem                  46851
window.Storage.setItem                  18104
window.Storage.removeItem               13812
CanvasRenderingContext2D.fill            7258
CanvasRenderingContext2D.save            7074
CanvasRenderingContext2D.restore         7070
HTMLCanvasElement.getContext             4208
window.Storage.key                       3689
CanvasRenderingContext2D.measureText     3103
CanvasRenderingContext2D.stroke          2393
CanvasRenderingContext2D.arc             1558
CanvasRenderingContext2D.fillRect        1371
CanvasRenderingContext2D.clip            1327
CanvasRenderingContext2D.fillText         858
CanvasRenderingContext2D.rotate           537
Name: symbol, dtype: int64

In [11]:
js[(js.operation == "call") &
   (js.symbol == "CanvasRenderingContext2D.fillText")
  ].arguments.value_counts().head(10)

{"0":"Cwm fjordbank glyphs vext quiz, 😃","1":4,"2":45}    74
{"0":"Cwm fjordbank glyphs vext quiz, 😃","1":2,"2":15}    74
{"0":"!image!","1":4,"2":17}                              39
{"0":"!image!","1":2,"2":15}                              39
{"0":"!H71JCaj)]# 1@#","1":4,"2":8}                       19
{"0":"Soft Ruddy Foothold 2","1":2,"2":2}                 19
{"0":"🇺🇳","1":0,"2":0}                                    18
{"0":"🇺​🇳","1":0,"2":0}                                   18
{"0":"🕴​♀️","1":0,"2":0}                                  14
{"0":"🕴‍♀️","1":0,"2":0}                                  14
Name: arguments, dtype: int64

<b> Criteria 3 from Englehardt & Narayanan, 2016 </b>

"3. The script should not call the save, restore, or addEventListener
methods of the rendering context."

In [12]:
CANVAS_FP_DO_NOT_CALL_LIST = ["CanvasRenderingContext2D.save",
                              "CanvasRenderingContext2D.restore",
                              "HTMLCanvasElement.addEventListener"]

In [387]:
# js[js.operation == "call"][js.arguments!=None].arguments.value_counts().head(15)
index_call_filter = js[js.symbol.isin(CANVAS_FP_DO_NOT_CALL_LIST)].index
index_call_filter

Int64Index([ 11423,  11426,  11427,  11428,  11429,  11430,  11431,  11432,
             11433,  11434,
            ...
            437057, 437058, 437059, 437060, 437061, 437062, 494490, 494495,
            494589, 494594],
           dtype='int64', length=14225)

<b> Criteria 1 from Englehardt & Narayanan, 2016: </b>

"1.The canvas element’s height and width properties must
not be set below 16 px."

In [114]:
def width_height_finder(data):
    w = re.compile('[^(line)]width', re.IGNORECASE)
    h = re.compile('[^(line)]height', re.IGNORECASE)
    ww = w.search(data)
    hh = h.search(data)
    if ww!= None:
        return data
    elif hh!=None:
        return data

In [115]:
l = list(map(width_height_finder, list(js[js.operation == "set"].symbol)))
l = [x for x in l if x is not None]

In [122]:
set(l)

{'HTMLCanvasElement.height', 'HTMLCanvasElement.width'}

In [360]:
MIN_CANVAS_IMAGE_WIDTH = 16
MIN_CANVAS_IMAGE_HEIGHT = 16


canvas_size = js[js.symbol.isin(["HTMLCanvasElement.width", "HTMLCanvasElement.height"])].value.astype(float)
index_size_filter = canvas_size.where(canvas_size < MIN_CANVAS_IMAGE_HEIGHT).index

In [381]:
index_size_filter

Int64Index([   638,    639,   1138,   1139,   1142,   1143,   1192,   1193,
              1241,   1242,
            ...
            501091, 501092, 501186, 501187, 501190, 501191, 501196, 501197,
            501200, 501201],
           dtype='int64', length=7879)

<b> Criteria 2 from Englehardt & Narayanan, 2016: </b> 

"2.Text must be written to canvas with least two colors or
at least 10 distinct characters."

In [167]:
js.symbol.value_counts()

window.document.cookie                               136037
window.navigator.userAgent                            85408
window.localStorage                                   48576
window.Storage.getItem                                46851
window.Storage.setItem                                18104
window.Storage.removeItem                             13812
window.name                                           13089
window.navigator.appName                              12283
window.sessionStorage                                 10357
CanvasRenderingContext2D.fillStyle                     8632
window.screen.colorDepth                               7806
CanvasRenderingContext2D.fill                          7258
CanvasRenderingContext2D.save                          7074
CanvasRenderingContext2D.restore                       7070
window.navigator.platform                              5187
window.navigator.appVersion                            4860
window.navigator.language               

In [ ]:
HTMLCanvasElement.style

In [170]:
js[js.symbol == "CanvasRenderingContext2D.strokeStyle"]

,id,crawl_id,visit_id,script_url,script_line,script_col,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp
1162,1163,3,3,https://static.xx.fbcdn.net/rsrc.php/v3iYXl4/yu/l/en_US/aArLzhwqJVj.js,19,2584,g.prototype.drawCircle,,https://www.facebook.com/,https://www.facebook.com/,,CanvasRenderingContext2D.strokeStyle,set,{},None,2018-06-27T14:19:44.150Z
1212,1213,3,3,https://static.xx.fbcdn.net/rsrc.php/v3iYXl4/yu/l/en_US/aArLzhwqJVj.js,19,2584,g.prototype.drawCircle,,https://www.facebook.com/,https://www.facebook.com/,,CanvasRenderingContext2D.strokeStyle,set,{},None,2018-06-27T14:19:44.194Z
1268,1269,3,3,https://static.xx.fbcdn.net/rsrc.php/v3iYXl4/yu/l/en_US/aArLzhwqJVj.js,19,2584,g.prototype.drawCircle,,https://www.facebook.com/,https://www.facebook.com/,,CanvasRenderingContext2D.strokeStyle,set,{},None,2018-06-27T14:19:44.270Z
1320,1321,3,3,https://static.xx.fbcdn.net/rsrc.php/v3iYXl4/yu/l/en_US/aArLzhwqJVj.js,19,2584,g.prototype.drawCircle,,https://www.facebook.com/,https://www.facebook.com/,,CanvasRenderingContext2D.strokeStyle,set,{},None,2018-06-27T14:19:44.316Z
20930,20931,4,72,https://code.createjs.com/createjs-2015.11.26.min.js,12,28940,b.exec,,https://s0.2mdn.net/2276943/1512572759808/index.html,https://www.cnn.com/,,CanvasRenderingContext2D.strokeStyle,set,#FFFFFF,None,2018-06-27T14:21:39.168Z
22016,22017,4,72,https://code.createjs.com/createjs-2015.11.26.min.js,12,28940,b.exec,,https://s0.2mdn.net/2276943/1512572759808/index.html,https://www.cnn.com/,,CanvasRenderingContext2D.strokeStyle,set,#FFFFFF,None,2018-06-27T14:21:40.144Z
23072,23073,4,72,https://code.createjs.com/createjs-2015.11.26.min.js,12,28940,b.exec,,https://s0.2mdn.net/2276943/1512572759808/index.html,https://www.cnn.com/,,CanvasRenderingContext2D.strokeStyle,set,#FFFFFF,None,2018-06-27T14:21:40.612Z
23641,23642,4,72,https://code.createjs.com/createjs-2015.11.26.min.js,12,28940,b.exec,,https://s0.2mdn.net/2276943/1512572759808/index.html,https://www.cnn.com/,,CanvasRenderingContext2D.strokeStyle,set,#FFFFFF,None,2018-06-27T14:21:40.794Z
23651,23652,4,72,https://code.createjs.com/createjs-2015.11.26.min.js,12,28940,b.exec,,https://s0.2mdn.net/2276943/1512572759808/index.html,https://www.cnn.com/,,CanvasRenderingContext2D.strokeStyle,set,#FFFFFF,None,2018-06-27T14:21:41.054Z
23906,23907,4,72,https://code.createjs.com/createjs-2015.11.26.min.js,12,28940,b.exec,,https://s0.2mdn.net/2276943/1512572759808/index.html,https://www.cnn.com/,,CanvasRenderingContext2D.strokeStyle,set,#FFFFFF,None,2018-06-27T14:21:45.126Z


In [350]:
import json 

def count_distinct_letters(text):
    if text!=None:
        t = json.loads(text, encoding='utf-8')['0']
        if not isinstance(t, int):
            return len(set(t))

In [ ]:
CANVAS_WRITE_FUNCS = [
    "CanvasRenderingContext2D.fillText",
    "CanvasRenderingContext2D.strokeText",
    "CanvasRenderingContext2D.fill"
    ]

In [398]:
text = js[js.symbol.isin(CANVAS_WRITE_FUNCS)].arguments.dropna()
text = map(count_distinct_letters, text)
res = pd.Series(text)
res = res.where(res<10)
index_count_filter = res.dropna().index
index_count_filter

Int64Index([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
            ...
            872, 873, 874, 875, 876, 877, 878, 879, 880, 881],
           dtype='int64', length=465)

<b>Criteria 4 from Englehardt & Narayanan, 2016: </b>

"4. The script extracts an image with toDataURL or with a
single call to getImageData that specifies an area with a
minimum size of 16px × 16px"

In [400]:
js.symbol.value_counts()

window.document.cookie                               136037
window.navigator.userAgent                            85408
window.localStorage                                   48576
window.Storage.getItem                                46851
window.Storage.setItem                                18104
window.Storage.removeItem                             13812
window.name                                           13089
window.navigator.appName                              12283
window.sessionStorage                                 10357
CanvasRenderingContext2D.fillStyle                     8632
window.screen.colorDepth                               7806
CanvasRenderingContext2D.fill                          7258
CanvasRenderingContext2D.save                          7074
CanvasRenderingContext2D.restore                       7070
window.navigator.platform                              5187
window.navigator.appVersion                            4860
window.navigator.language               

In [447]:
CANVAS_READ_FUNCS = [
    "HTMLCanvasElement.toDataURL",
    "CanvasRenderingContext2D.getImageData"
    ]

In [448]:
js[js.symbol.isin(CANVAS_READ_FUNCS)].arguments.dropna()

3221                  {"0":"image/webp"}
6203                  {"0":"image/webp"}
6208                  {"0":"image/webp"}
6401                  {"0":"image/webp"}
20885          {"0":0,"1":0,"2":1,"3":1}
30687          {"0":0,"1":0,"2":1,"3":1}
39266          {"0":0,"1":0,"2":1,"3":1}
39275          {"0":0,"1":0,"2":1,"3":1}
39591                  {"0":"image/png"}
39594                  {"0":"image/png"}
39597                  {"0":"image/png"}
39600                  {"0":"image/png"}
39603                  {"0":"image/png"}
39606                  {"0":"image/png"}
39609                  {"0":"image/png"}
39612                  {"0":"image/png"}
40619                  {"0":"image/png"}
43951                 {"0":"image/webp"}
44877          {"0":0,"1":0,"2":1,"3":1}
55807        {"0":0,"1":0,"2":32,"3":32}
56261       {"0":0,"1":0,"2":1024,"3":1}
56262       {"0":0,"1":0,"2":1024,"3":1}
56264       {"0":0,"1":0,"2":1024,"3":1}
56267       {"0":0,"1":0,"2":1024,"3":1}
56268       {"0"

In [449]:
def check_dimensions(image):
    if image!=None:
        img = json.loads(image, encoding='utf-8')
        if len(img)>3 and img['2']>16 and img['3']>16:
            return image

In [450]:
image = js[js.symbol.isin(CANVAS_READ_FUNCS)].arguments.dropna()
image = map(check_dimensions, image)
res = pd.Series(image)
index_image_filter = res.dropna().index
index_image_filter

Int64Index([ 19,  37,  44,  45,  49,  50,  51,  52,  53,  54,
            ...
            261, 262, 263, 264, 265, 266, 267, 268, 269, 285],
           dtype='int64', length=148)

In [468]:
lst = []

In [469]:
for i in range(len(js)):
    if i in index_call_filter and i in index_count_filter:
        lst.append(js.iloc[i])
    elif i in index_call_filter and i in index_size_filter:
        lst.append(js.iloc[i])
    elif i in index_call_filter and i in index_image_filter:
        lst.append(js.iloc[i])
    elif i in index_count_filter and i in index_size_filter:
        lst.append(js.iloc[i])
    elif i in index_count_filter and i in index_image_filter:
        lst.append(js.iloc[i])
    elif i in index_image_filter and i in index_size_filter:
        lst.append(js.iloc[i])

In [471]:
df = pd.DataFrame(lst)
df

,id,crawl_id,visit_id,script_url,script_line,script_col,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp
56,57,11,11,"https://www.gstatic.com/og/_/js/k=og.og2.en_US.6t-3CYB7djQ.O/rt=j/m=def/exm=in,fot/d=1/ed=1/rs=AA2YrTvhjyOTalwzfqFzbvVdBxFa9GK0UQ",172,315,uu,,https://www.google.co.jp/?gws_rd=ssl,https://www.google.co.jp/?gws_rd=ssl,,window.localStorage,get,{},None,2018-06-27T14:19:41.046Z
57,58,5,5,https://www.wikipedia.org/portal/wikipedia.org/assets/js/index-c56562e385.js,1,7748,mw.storage.get,,https://www.wikipedia.org/,https://www.wikipedia.org/,,window.Storage.getItem,call,,"{""0"":""portal_test_group_expires""}",2018-06-27T14:19:41.092Z
58,59,11,11,"https://www.gstatic.com/og/_/js/k=og.og2.en_US.6t-3CYB7djQ.O/rt=j/m=def/exm=in,fot/d=1/ed=1/rs=AA2YrTvhjyOTalwzfqFzbvVdBxFa9GK0UQ",174,106,tu.prototype.b,,https://www.google.co.jp/?gws_rd=ssl,https://www.google.co.jp/?gws_rd=ssl,,window.localStorage,get,{},None,2018-06-27T14:19:41.046Z
59,60,11,11,"https://www.gstatic.com/og/_/js/k=og.og2.en_US.6t-3CYB7djQ.O/rt=j/m=def/exm=in,fot/d=1/ed=1/rs=AA2YrTvhjyOTalwzfqFzbvVdBxFa9GK0UQ",174,113,tu.prototype.b,,https://www.google.co.jp/?gws_rd=ssl,https://www.google.co.jp/?gws_rd=ssl,,window.Storage.getItem,call,,"{""0"":""og-up-OGPL""}",2018-06-27T14:19:41.046Z
110,111,11,11,"https://apis.google.com/_/scs/abc-static/_/js/k=gapi.gapi.en.yK0z3MKtgaU.O/m=gapi_iframes,googleapis_client,plusone/rt=j/sv=1/d=1/ed=1/rs=AHpOoo-SafOYj4n3budMysbWxppU-lxJeg/cb=gapi.loaded_0",345,15,,,https://www.google.co.jp/?gws_rd=ssl,https://www.google.co.jp/?gws_rd=ssl,,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,None,2018-06-27T14:19:41.328Z
111,112,11,11,"https://apis.google.com/_/scs/abc-static/_/js/k=gapi.gapi.en.yK0z3MKtgaU.O/m=gapi_iframes,googleapis_client,plusone/rt=j/sv=1/d=1/ed=1/rs=AHpOoo-SafOYj4n3budMysbWxppU-lxJeg/cb=gapi.loaded_0",234,46,_.jf,,https://www.google.co.jp/?gws_rd=ssl,https://www.google.co.jp/?gws_rd=ssl,,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,None,2018-06-27T14:19:41.338Z
112,113,11,11,"https://apis.google.com/_/scs/abc-static/_/js/k=gapi.gapi.en.yK0z3MKtgaU.O/m=gapi_iframes,googleapis_client,plusone/rt=j/sv=1/d=1/ed=1/rs=AHpOoo-SafOYj4n3budMysbWxppU-lxJeg/cb=gapi.loaded_0",234,46,_.jf,,https://www.google.co.jp/?gws_rd=ssl,https://www.google.co.jp/?gws_rd=ssl,,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,None,2018-06-27T14:19:41.338Z
115,116,11,11,"https://www.google.co.jp/xjs/_/js/k=xjs.s.en_US.EJn3JxysB74.O/m=sx,sb,cdos,cr,elog,hsm,jsa,r,d,csi/am=IIvIbAwBkP-nYCBRwApGWGACBUME/rt=j/d=1/dg=0/rs=ACT90oGqcE1bZa7UbFe0NMwBT2f0wRTwJQ",216,623,s_.get,,https://www.google.co.jp/?gws_rd=ssl,https://www.google.co.jp/?gws_rd=ssl,,window.Storage.getItem,call,,"{""0"":""jsRedirect;;nvstrt""}",2018-06-27T14:19:41.376Z
116,117,11,11,"https://www.google.co.jp/xjs/_/js/k=xjs.s.en_US.EJn3JxysB74.O/m=sx,sb,cdos,cr,elog,hsm,jsa,r,d,csi/am=IIvIbAwBkP-nYCBRwApGWGACBUME/rt=j/d=1/dg=0/rs=ACT90oGqcE1bZa7UbFe0NMwBT2f0wRTwJQ",216,623,s_.get,,https://www.google.co.jp/?gws_rd=ssl,https://www.google.co.jp/?gws_rd=ssl,,window.Storage.getItem,call,,"{""0"":""jsRedirect;;nvstrt""}",2018-06-27T14:19:41.378Z
125,126,13,13,https://r1.res.office365.com/owalanding/v2.8/landing.js,32,36246,"[""./components/App.tsx""]/O</t.prototype.detectEdge",,https://outlook.live.com/owa/,https://outlook.live.com/owa/,,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,None,2018-06-27T14:19:40.922Z
